In [1]:
import pandas as pd
import numpy as np
import glob
import plotly.graph_objects as go

In [2]:
# functions written by Mitali and Girish in 2024
# adapted by Tanvi Deora, April 2025

def theta_phi(x1, y1, z1, x2, y2, z2, x3, y3, z3, x4, y4, z4):
    """
    Calculate the angles theta and phi between two vectors

    Parameters:
        x1,y1,z1 (float) : The coordinates of point 1.
        x2,y2,z2 (float) : The coordinates of point 2.
        x3,y3,z3 (float) : The coordinates of point 3.
        x4,y4,z4 (float) : The coordinates of point 4.

    Returns:
        theta (float) : The angle (in degrees) between point1, point 2 and point3 in the XY plane.
        phi (float) : The angle (in degrees) between point1, point 2 and point3 in the ZY plane.

    Dependencies:
        - numpy (np): Required for numerical computations and array operations.
    """

    # Convert the input coordinates to NumPy arrays for vector operations
    a = np.array([x1,y1,z1]) # head
    b = np.array([x2,y2,z2]) # abdomen
    c = np.array([x3,y3,z3]) # COM of leading (chasee) fly
    d = np.array([x4,y4,z4]) # COM of lagging (chaser) fly
    

    # Calculate the vectors
    ab = a - b # changed this from ba = a -b for consistency 
    cd = c - d 

    # Calculate the theta angle (angle in XY plane)
    theta = np.degrees(np.arctan2(np.cross(ab[:2], cd[:2]), np.dot(ab[:2], cd[:2])))

    # Calculate the phi angle (angle in ZY plane) 
    phi = np.degrees(np.arctan2(np.cross(ab[[0, 2]], cd[[0, 2]]), np.dot(ab[[0, 2]], cd[[0, 2]])))

    return theta, phi

In [4]:
files = glob.glob('./Data/Outputs/Sampled_1k/COM/*_smoothened_*COM.csv')
files

['./Data/Outputs/Sampled_1k/COM\\2023-04-19_Trial7_Set1_5k_smoothened_interpol_COM.csv',
 './Data/Outputs/Sampled_1k/COM\\2023-04-21_Trial6_1k_smoothened_interpol_COM.csv',
 './Data/Outputs/Sampled_1k/COM\\2023-04-21_Trial7_1k_smoothened_interpol_COM.csv',
 './Data/Outputs/Sampled_1k/COM\\2023-04-28_Trial4_5k_smoothened_interpol_COM.csv',
 './Data/Outputs/Sampled_1k/COM\\2023-04-28_Trial5_5k_smoothened_interpol_COM.csv',
 './Data/Outputs/Sampled_1k/COM\\2023-04-28_Trial8_1k_smoothened_interpol_COM.csv']

In [4]:
info = pd.read_csv('./Data/Outputs/Raw_xyz_points/metadata.csv')
info

,filename,start,end,framerate,chase_type,Chaser,Chasee,Notes
0,2023-04-19_Trial7_Set1_5k_xyzpts.csv,487.0,926.0,5000,Intact male chasing female,I_M,I_F,Perfect
1,2023-04-19_Trial8_Set1_5k_xyzpts.csv,NaN,NaN,5000,Intact male chasing male,I_M,I_M,80% digitized. Needs to fill some gaps
2,2023-04-21_Trial5_1k_xyzpts.csv,115.0,283.0,1000,Intact male chasing ablated male,I_M,A_M,Can't find the project file. Only the xyz pts ...
3,2023-04-21_Trial6_1k_xyzpts.csv,56.0,276.0,1000,Intact male chasing intact male,I_M,I_M,Perfect
4,2023-04-21_Trial7_1k_xyzpts.csv,112.0,703.0,1000,Intact male chasing intact male,I_M,I_M,Perfect
5,2023-04-28_Trial4_5k_xyzpts.csv,332.0,843.0,5000,Ablated male chasing intact male,A_M,I_M,Perfect
6,2023-04-28_Trial5_5k_xyzpts.csv,1420.0,2510.0,5000,Ablated male chasing intact male,A_M,I_M,Perfect
7,2023-04-28_Trial6_5k_xyzpts.csv,58.0,1164.0,5000,Ablated male chasing intact male,A_M,I_M,Can't find the project file. Only the xyz pts ...
8,2023-04-28_Trial8_1k_xyzpts.csv,108.0,276.0,1000,Intact male chasing ablated male,I_M,A_M,Perfect


In [5]:
for f in files:
    name = f.split('\\')[-1][:-4]
    df = pd.read_csv(f)
    
    framerate = 1000 # for sequences sampled at 1k
#     framerate = [row['framerate'] for ind, row in info.iterrows() if row['filename'][:-10] in f.split('\\')[-1]][0] 
    
    # get points to compute error angles
    # point connecting the the chaser and chasee
    chasee_COM = df.loc[:, ['chasee_com_x', 'chasee_com_y', 'chasee_com_z']]
    chaser_COM = df.loc[:, ['chaser_com_x', 'chaser_com_y', 'chaser_com_z']]
    
    #heading vector for chaser
    chaser_head = df.loc[:, ['chaser_head_X', 'chaser_head_Y', 'chaser_head_Z']]
    chaser_abdo = df.loc[:,['chaser_abdomen_X', 'chaser_abdomen_Y', 'chaser_abdomen_Z']]
    
    for all_vectors in [chasee_COM, chaser_COM, chaser_head, chaser_abdo]:
        print(all_vectors.isnull().values.any())
    
        
    chase_theta = []
    chase_phi = []
    for ind, _ in chasee_COM.iterrows():
        pt3 = chasee_COM.loc[ind, :]
        pt4 = chaser_COM.loc[ind, :]
        
        pt1 = chaser_head.loc[ind, :]
        pt2 = chaser_abdo.loc[ind, :]
        
        tt, ph = theta_phi(
            pt1['chaser_head_X'], pt1['chaser_head_Y'], pt1['chaser_head_Z'],
            pt2['chaser_abdomen_X'], pt2['chaser_abdomen_Y'], pt2['chaser_abdomen_Z'],
            pt3['chasee_com_x'], pt3['chasee_com_y'], pt3['chasee_com_z'],
            pt4['chaser_com_x'], pt4['chaser_com_y'], pt4['chaser_com_z']
        )
        
           
        chase_theta.append(tt)
        chase_phi.append(ph)
        
       
    
    #Calculating angular velocity of Chasee and chaser
    angvel_chase_theta = np.diff(chase_theta)*framerate
    angvel_chase_phi = np.diff(chase_phi)*framerate
    angvel_chase_theta = np.append(angvel_chase_theta, np.repeat(np.nan, 1))
    angvel_chase_phi = np.append(angvel_chase_phi, np.repeat(np.nan, 1))
    
    #Calculating angular accerelation of Chasee and chaser
    angacc_chase_theta = np.diff(angvel_chase_theta)
    angacc_chase_phi = np. diff(angvel_chase_phi)
    angacc_chase_theta = np.append(angacc_chase_theta, np.repeat(np.nan, 1))
    angacc_chase_phi = np.append(angacc_chase_phi, np.repeat(np.nan, 1))
  
    
    chase_df = pd.DataFrame({'chase_theta': chase_theta,
                            'chase_phi': chase_phi,
                            'chase_theta_speed': angvel_chase_theta,
                             'chase_phi_speed': angvel_chase_phi,
                             'chase_theta_acc': angacc_chase_theta,
                             'chase_phi_acc': angacc_chase_phi,
                            })
    chase_df.to_csv('./Data/Outputs/Sampled_1k/Chase/' + name + '_ChaseErrorAnglesSpeedAcc.csv')

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
